In [1]:
#system libraries
import os
import random
import time
#selenium libraries
#from webdriver_manager.chrome import ChromeDriverManager # to download driver automatic
# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.chrome.options import Options
# to find links
from bs4 import BeautifulSoup
import json
import requests
import urllib.request
import re
import time # to sleep
#recaptcha libraries, can be found on pip install
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub 
# to output a dataframe to file 
import pandas as pd
# fill this in with your job preferences!
# glassdoor search bar 
PREFERENCES = {
    "position_title": "Entwicklungsingenieur",
    "location": "Berlin, Deutschland"
}
# glassdoor login credentiels
from secret import glassdoor_credentials as s
LOGIN = {
    "username": s['email'],  # to be changed with the email
    "password": s['password'] # to be changed with the password
}
JOB_APP = {
    "first_name": "Sami",
    "last_name": "Dhiab",
    "fullname": "Sami Dhiab",
    "email": s['email'],
    "phone": "+4917666994604",
    "title": "M.Eng.Robotics",
    "company": "DLR",
    "resume": "C:/Users/Sami/Downloads/Resume-Sami-Dhiab.pdf", # change path to resume, for windows C:\\ should be present
    "linkedin": "https://www.linkedin.com/",
    "website": "www.youtube.com",
    "github": "https://github.com", 
    "twitter": "www.twitter.com",
    "city": "Berlin",
    "country": "Deutschland",
    "grad_month": '05',
    "grad_year": '2021',
    "university": "Beuth" 
}


C:\Python39\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# default selenium configuartions
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
#options.add_argument("--headless") # if headless mode
options.add_argument("user-data-dir=selenium") # to store cookies between sessions
driver = webdriver.Chrome(options=options, executable_path='C:/Users/Sami/chromedriver.exe') # change path to chromedriver.exe


In [3]:
def delay ():
    time.sleep(random.randint(2,3))
 
try:    # if first time login
    # browse the page and retrieve search resultst
    driver.get('https://www.glassdoor.de/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK')
    # fill out username and password
    driver.find_element_by_xpath("//*[@id='userEmail']").send_keys(LOGIN['username']) 
    driver.find_element_by_xpath("//*[@id='userPassword']").send_keys(LOGIN['password'])
    # accept cookies and click login
    driver.find_element_by_xpath('//button[text()="Cookies akzeptieren"]').click()
    driver.find_element_by_xpath('//button[text()="Anmelden"]').click()
    # keep waiting for user to log-in until the URL changes to user page
    while True:
        try:
            WebDriverWait(driver, 5).until(EC.url_contains("member"))
        except TimeoutException:
            break
except:
    print('you are already logged in')
    driver.get('https://www.glassdoor.de/')

    pass
# wait for the search bar to appear
try:
    # look for search bar fields and send elemnets
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='scBar']")))
    keyword=driver.find_element_by_xpath("//*[@id='sc.keyword']")
    keyword.send_keys(Keys.CONTROL + "a")
    keyword.send_keys(Keys.DELETE)
    keyword.send_keys(PREFERENCES['position_title']) # type in search bar
    location=driver.find_element_by_xpath("//*[@id='sc.location']")
    location.send_keys(Keys.CONTROL + "a")
    location.send_keys(Keys.DELETE)
    location.send_keys(PREFERENCES['location']) # type in location bar
    driver.find_element_by_xpath(" //*[@id='scBar']/div/button").click() # click search

except NoSuchElementException:
    print('check unfoudn elements')
# filter jobs by quick apply jobs
try:
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="DKFilters"]')))
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[1]/span[1]').click()
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[2]/div[10]/label').click()
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[1]/span[1]').click()
except ElementClickInterceptedException:
    pass

you are already logged in


In [4]:
# collect all links from current page using bs
time.sleep(1)
url = driver.current_url
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=hdr)
infile = urllib.request.urlopen(req).read()
response = infile.decode('ISO-8859-1')
soup = BeautifulSoup(response)
# find all hrefs
allJobLinks = soup.findAll("a", {"class": "jobLink"})
allLinks = list(set([jobLink['href'] for jobLink in allJobLinks]))
#print(allLinks[0])
#print(type(allLinks[0]))
print("number of job found", len(allLinks))

number of job found 12


In [31]:
#recaptcha libraries, can be found on pip install
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub 
# iterarte all links to apply
try:
    for link in allLinks:
        url = "https://www.glassdoor.de"+link
        print("applying for the current jobpage: ", url)
        print()
        driver.get(url)
        # bypass popup if applying on main page
        try:
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span/svg').click()
        except NoSuchElementException:
            pass 
        # if redirecting to the jobpage 
        driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div[1]/span/button').click() # click schnellbewerbung
        try:
            myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//iframe[@title='Job application form container']")))
            print("Page is ready!")
        except TimeoutException:
            print("Loading took too much time!, skipping job")
            continue
        driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='Job application form container']")) #switch to integrated frame 1
        driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='Job application form']")) #switch to second iframe
        # trying differents forms
        try:
            pop_up=driver.find_element_by_xpath('//*[@id="close-popup"]')
            if pop_up.is_displayed():
                print("popup found")
            else:
                print("popup not found")
            pop_up.click() #close popup if already apllied for job
            print('already applied, closing popup')
            continue
        except NoSuchElementException:
            print('no popup, applying..')
            
        try:
            driver.find_element_by_xpath('//*[@id="input-applicant.name"]').send_keys(JOB_APP['fullname'])
            driver.find_element_by_xpath('//*[@id="input-applicant.email"]').send_keys(JOB_APP['email'])
            driver.find_element_by_xpath('//*[@id="input-applicant.phoneNumber"]').send_keys(JOB_APP['phone'])
            driver.find_element_by_xpath('//*[@id="input-applicant.location.city"]').send_keys(JOB_APP['city'])
            driver.find_element_by_xpath('//*[@id="input-applicant.jobTitle"]').send_keys(JOB_APP['title'])
            driver.find_element_by_xpath('//*[@id="input-applicant.companyName"]').send_keys(JOB_APP['company'])
        except NoSuchElementException:
            pass
        else:
            driver.find_element_by_xpath('//*[@id="input-applicant.firstName"]').send_keys(JOB_APP['first_name'])
            driver.find_element_by_xpath('//*[@id="input-applicant.lastName"]').send_keys(JOB_APP['last_name'])
            driver.find_element_by_xpath('//*[@id="input-applicant.email"]').send_keys(JOB_APP['email'])
            driver.find_element_by_xpath('//*[@id="input-applicant.phoneNumber"]').send_keys(JOB_APP['phone'])
            driver.find_element_by_xpath('//*[@id="input-applicant.location.city"]').send_keys(JOB_APP['city'])
            driver.find_element_by_xpath('//*[@id="input-applicant.jobTitle"]').send_keys(JOB_APP['title'])
            driver.find_element_by_xpath('//*[@id="input-applicant.companyName"]').send_keys(JOB_APP['company'])
        finally: 
            driver.find_element_by_name('applicant.fileUpload').send_keys(JOB_APP['resume']) #upload resume
            #locate weiter button and click it
            weiter = driver.find_element_by_xpath('//*[@id="form-action-continue"]') 
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="form-action-continue"]'))).click()
            weiter.click()
            weiter.submit()
            if weiter.is_displayed():
                print("Element found")
            else:
                print("Element not found")
            delay()
            #ja click
            try:
                driver.find_element_by_xpath('//*[@id="radio-option-0"]').click()
                #locate weiter button and click it
                weiter = driver.find_element_by_xpath('//*[@id="form-action-continue"]') 
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="form-action-continue"]'))).click()
                weiter.click()
                weiter.submit()
                if weiter.is_displayed():
                    print("Element found")
                else:
                    print("Element not found")
                delay()
            except NoSuchElementException:
                pass

            #switch to recaptcha frame
            frames=driver.find_elements_by_tag_name("iframe")
            print(len(frames))
            #driver.switch_to.frame(frames[0])
            driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="recaptcha-applyForm"]/div/div/iframe'))
            delay()
            #click on checkbox to activate recaptcha
            driver.find_element_by_class_name("recaptcha-checkbox-border").click()
            delay()
            #switch to recaptcha audio control frame
            #driver.switch_to.default_content()
            #frames=driver.find_element_by_xpath("/html/body/div[2]/div[4]/iframe")
            #driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='reCAPTCHA-Aufgabe']"))
            driver.implicitly_wait(10) # seconds
            driver.switch_to.parent_frame()
            #driver.switch_to.frame(frames[1])
            driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='reCAPTCHA-Aufgabe']"))
            delay()
            #click on audio challenge
            driver.find_element_by_id("recaptcha-audio-button").click()
            delay()
            #switch to recaptcha audio challenge frame
            #driver.switch_to.default_content()
            #driver.switch_to.parent_frame()
            #frames= driver.find_elements_by_tag_name("iframe")
            #driver.switch_to.frame(frames[-1])
            #delay()
            #click on the play button
            driver.find_element_by_xpath('//*[@id=":2"]').click()
            #get the mp3 audio file
            src = driver.find_element_by_id("audio-source").get_attribute("src")
            print("[INFO] Audio src: %s"%src)
            #download the mp3 audio file from the source
            urllib.request.urlretrieve(src, os.path.normpath(os.getcwd()+"\\sample.mp3"))
            delay()
            #load downloaded mp3 audio file as .wav
            try:
                sound = pydub.AudioSegment.from_mp3(os.path.normpath(os.getcwd()+"\\sample.mp3"))
                sound.export(os.path.normpath(os.getcwd()+"\\sample.wav"), format="wav")
                sample_audio = sr.AudioFile(os.path.normpath(os.getcwd()+"\\sample.wav"))
            except:
                print("[-] Please run program as administrator")
                
            r= sr.Recognizer()

            with sample_audio as source:
                audio = r.record(source)

            #translate audio to text with google voice recognition
            key=r.recognize_google(audio) #,language='en-EN'
            print("[INFO] Recaptcha Passcode: %s"%key)

            #key in results and submit
            #try to imitate human typing
            for letter in key:
                driver.find_element_by_id("audio-response").send_keys(letter.lower())
                time.sleep(2)
            
            driver.find_element_by_id("audio-response").send_keys(Keys.ENTER)
            driver.switch_to.default_content()
            delay()
            driver.find_element_by_id("recaptcha-demo-submit").click()
            delay()
            driver.find_element_by_id("form-action-submit").click()
except ElementNotVisibleException:

    driver.close()
    pass

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.htm?pos=107&ao=1136006&s=58&guid=0000017889f3c761a85b1d93363478ea&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_f69e2dce&cb=1617222158349&jobListingId=3797109513

Page is ready!
no popup, applying..
Element found
3
[INFO] Audio src: https://www.google.com/recaptcha/api2/payload?p=06AGdBq26Nfbrb-NT-l02lJ5YNDKy_9kmiZQhSP5XE9kkzyR5JM4hUzgxvmaLOxm851LespcSr8lvwOX58WpY82UoLLogJ39rBOXAj_wULj3iUv-xV3kVbzqYKBpnKiBVKOcjIj678ZAyXUeYv7-2RKJrnpCDA9Rcj4BX4H_dfQmZXH0OmMCO9zjNHtOk55sLwQGTt4pPxboic&k=6LcW5DYUAAAAACmDnrXUOb4TWEJGZkhsE6MWTDIG
[INFO] Recaptcha Passcode: a business checkout


KeyboardInterrupt: 